In [28]:
import lightgbm as lgb

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

In [2]:
df_train = pd.read_csv('train.csv')

In [3]:
df_train.head()

,TXKEY,Time,V1,V2,V3,V4,V5,V6,V7,V8,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,T43428,41505.0,-16.526507,8.584972,-18.649853,9.505594,-13.793819,-2.832404,-16.701694,7.517344,...,1.190739,-1.127670,-2.358579,0.673461,-1.413700,-0.462762,-2.018575,-1.042804,364.19,1
1,T49906,44261.0,0.339812,-2.743745,-0.134070,-1.385729,-1.451413,1.015887,-0.524379,0.224060,...,-0.213436,-0.942525,-0.526819,-1.156992,0.311211,-0.746647,0.040996,0.102038,520.12,0
2,T29474,35484.0,1.399590,-0.590701,0.168619,-1.029950,-0.539806,0.040444,-0.712567,0.002299,...,0.102398,0.168269,-0.166639,-0.810250,0.505083,-0.232340,0.011409,0.004634,31.00,0
3,T276481,167123.0,-0.432071,1.647895,-1.669361,-0.349504,0.785785,-0.630647,0.276990,0.586025,...,0.358932,0.873663,-0.178642,-0.017171,-0.207392,-0.157756,-0.237386,0.001934,1.50,0
4,T278846,168473.0,2.014160,-0.137394,-1.015839,0.327269,-0.182179,-0.956571,0.043241,-0.160746,...,-0.238644,-0.616400,0.347045,0.061561,-0.360196,0.174730,-0.078043,-0.070571,0.89,0


In [4]:
x = df_train.drop(['TXKEY','Class'],axis=1)
y = df_train['Class']

In [39]:
df_test = pd.read_csv('test_public.csv')

In [40]:
TXKEY =np.array(df_test["TXKEY"])
TXKEY

array(['T2', 'T5', 'T7', ..., 'T284797', 'T284800', 'T284802'],
      dtype=object)

In [41]:
X_test = df_test.drop(['TXKEY'],axis=1)

In [59]:
def LightGBM_GBDT_plus_LR(x,y):
    
    predictions = 0
    
    for i in range(5):
        
        # create dataset for lightgbm
        lgb_train = lgb.Dataset(x,y)

        params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': {'binary_logloss'},
        'num_leaves': 64,
        'num_trees': 100,
        'learning_rate': 0.01,
        'feature_fraction': 0.9,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'verbose': 0
        }

        # number of leaves,will be used in feature transformation
        num_leaf = 64

        print('Start training...')
        # train
        gbm = lgb.train(params,
                        lgb_train,
                        num_boost_round=300,
                        valid_sets=lgb_train)

        print('Save model...')
        # save model to file
        gbm.save_model('model.txt')

        print('Start predicting...')
        # predict and get data on leaves, training data
        y_pred = gbm.predict(x, pred_leaf=True)

        print('Writing transformed training data')
        transformed_training_matrix = np.zeros([len(y_pred), len(y_pred[0]) * num_leaf],dtype=np.int64)  # N * num_tress * num_leafs
        for i in range(0, len(y_pred)):
            temp = np.arange(len(y_pred[0])) * num_leaf + np.array(y_pred[i])
            transformed_training_matrix[i][temp] += 1

        lr = LogisticRegression(penalty='l2',C=0.05) # logistic model construction
        lr.fit(transformed_training_matrix,y)  # fitting the data
        
        y_pred = gbm.predict(X_test, pred_leaf=True)
        
        transformed_testing_matrix = np.zeros([len(y_pred), len(y_pred[0]) * num_leaf], dtype=np.int64)
        for i in range(0, len(y_pred)):
            temp = np.arange(len(y_pred[0])) * num_leaf + np.array(y_pred[i])
            transformed_testing_matrix[i][temp] += 1
        
        y_pred_test = lr.predict(transformed_testing_matrix)   # Give the probabilty on each label
        
        if i == 0:
            predictions = y_pred_test
        else:
            predictions += y_pred_test
            
    for j in range(len(predictions)):
        if predictions[j] >= 5:
            predictions[j] = 1
        else:
            predictions[j] = 0
    
    my_solution = pd.DataFrame(predictions, TXKEY, columns = ["Class"])
    my_solution
    
    my_solution.to_csv("Ans/solution_17.csv", index_label = ["TXKEY"])

In [60]:
LightGBM_GBDT_plus_LR(x,y)

Start training...


/Users/llc/anaconda2/envs/python3/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	training's binary_logloss: 0.00608694
[2]	training's binary_logloss: 0.00578185
[3]	training's binary_logloss: 0.00555798
[4]	training's binary_logloss: 0.00540876
[5]	training's binary_logloss: 0.00525913
[6]	training's binary_logloss: 0.00512439
[7]	training's binary_logloss: 0.00500932
[8]	training's binary_logloss: 0.00489742
[9]	training's binary_logloss: 0.00479285
[10]	training's binary_logloss: 0.00469912
[11]	training's binary_logloss: 0.00459227
[12]	training's binary_logloss: 0.00449933
[13]	training's binary_logloss: 0.00441153
[14]	training's binary_logloss: 0.00432356
[15]	training's binary_logloss: 0.00424865
[16]	training's binary_logloss: 0.00415845
[17]	training's binary_logloss: 0.00407362
[18]	training's binary_logloss: 0.00400211
[19]	training's binary_logloss: 0.00393314
[20]	training's binary_logloss: 0.00386861
[21]	training's binary_logloss: 0.00380176
[22]	training's binary_logloss: 0.0037397
[23]	training's binary_logloss: 0.00368197
[24]	training's binar

[91]	training's binary_logloss: 0.00165031
[92]	training's binary_logloss: 0.00163416
[93]	training's binary_logloss: 0.00161803
[94]	training's binary_logloss: 0.00160226
[95]	training's binary_logloss: 0.00158654
[96]	training's binary_logloss: 0.00156987
[97]	training's binary_logloss: 0.00155355
[98]	training's binary_logloss: 0.00153747
[99]	training's binary_logloss: 0.00152166
[100]	training's binary_logloss: 0.00150615
Save model...
Start predicting...
Writing transformed training data
Start training...
[1]	training's binary_logloss: 0.00608694
[2]	training's binary_logloss: 0.00578185
[3]	training's binary_logloss: 0.00555798
[4]	training's binary_logloss: 0.00540876
[5]	training's binary_logloss: 0.00525913
[6]	training's binary_logloss: 0.00512439
[7]	training's binary_logloss: 0.00500932
[8]	training's binary_logloss: 0.00489742
[9]	training's binary_logloss: 0.00479285
[10]	training's binary_logloss: 0.00469912
[11]	training's binary_logloss: 0.00459227
[12]	training's bin

[80]	training's binary_logloss: 0.00184165
[81]	training's binary_logloss: 0.00182295
[82]	training's binary_logloss: 0.00180473
[83]	training's binary_logloss: 0.00178667
[84]	training's binary_logloss: 0.00176908
[85]	training's binary_logloss: 0.00175172
[86]	training's binary_logloss: 0.00173391
[87]	training's binary_logloss: 0.00171664
[88]	training's binary_logloss: 0.00169978
[89]	training's binary_logloss: 0.00168303
[90]	training's binary_logloss: 0.00166664
[91]	training's binary_logloss: 0.00165031
[92]	training's binary_logloss: 0.00163416
[93]	training's binary_logloss: 0.00161803
[94]	training's binary_logloss: 0.00160226
[95]	training's binary_logloss: 0.00158654
[96]	training's binary_logloss: 0.00156987
[97]	training's binary_logloss: 0.00155355
[98]	training's binary_logloss: 0.00153747
[99]	training's binary_logloss: 0.00152166
[100]	training's binary_logloss: 0.00150615
Save model...
Start predicting...
Writing transformed training data
Start training...
[1]	traini